In [1]:
#import Dependencies
import pandas as pd
import numpy as np
import tweepy
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import re
import csv
from textblob import TextBlob

In [2]:
from config import (consumer_key,consumer_secret,access_token,access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [3]:
# Target Search Term
target_terms = ("@EverydayHealth", "@top_tw_health", "@HealthCareGov",
                "@abchealthin","@CDCemergency","@CDCFlu","@CDCgov")


In [4]:
# Counter
# Variables for holding sentiments
sentiments = []

# Loop through all target users
for target in target_terms:
    counter = 1
#Loop through 100 tweets
    for x in range(5):

    #Get tweets from homefeed
        public_tweets = api.user_timeline(target, page=x,count=200)


        for tweet in public_tweets:
            compound = analyzer.polarity_scores(tweet["text"])["compound"]
            pos = analyzer.polarity_scores(tweet["text"])["pos"]
            neu = analyzer.polarity_scores(tweet["text"])["neu"]
            neg = analyzer.polarity_scores(tweet["text"])["neg"]
            text=tweet['text']
            date=tweet['created_at']


 # Add sentiments for each tweet into a list
            sentiments.append({"Tweets":text,
                       "Positive": pos,
                       "Negative": neg})

# Add to counter 
            counter += 1


In [5]:
# Convert sentiments to DataFrame
sentiments_pd = pd.DataFrame.from_dict(sentiments)
sentiments_pd.shape


(6946, 3)

In [6]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [7]:
sentiments_pd['tidy_tweet'] = np.vectorize(remove_pattern)(sentiments_pd['Tweets'], "@[\w]*") 
sentiments_pd.shape

(6946, 4)

In [8]:
def f(row):
    if (row['Positive']>row['Negative']):
        val = "Positive"
        
    elif (row['Positive']<row['Negative']):
        val = "Negative"
    else:
        val = "Neutral"
    return val

In [9]:
sentiments_pd['Sentiment'] = sentiments_pd.apply(f, axis=1)


In [10]:
sentiments_pd['tidy_tweet'] = sentiments_pd['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")

In [11]:
sentiments_pd

,Negative,Positive,Tweets,tidy_tweet,Sentiment
0,0.000,0.132,Human urine has been a useful diagnostic tool ...,Human urine has been a useful diagnostic tool ...,Positive
1,0.000,0.000,This is what it means if you carry extra weigh...,This is what it means if you carry extra weigh...,Neutral
2,0.000,0.000,5 reasons to follow this plan to demolish bell...,reasons to follow this plan to demolish bell...,Neutral
3,0.000,0.426,9 natural ways to boost your sex life: https:/...,natural ways to boost your sex life https ...,Positive
4,0.000,0.000,The one thing you need to get rid of adult acn...,The one thing you need to get rid of adult acn...,Neutral
5,0.000,0.000,This is what happens to your body when you eat...,This is what happens to your body when you eat...,Neutral
6,0.000,0.000,"Swelling in your eyes, canker sores, and stiff...",Swelling in your eyes canker sores and stiff...,Neutral
7,0.000,0.000,This is what happens to your body when you hav...,This is what happens to your body when you hav...,Neutral
8,0.000,0.181,Too cold to run outside? Here's the 10 best wa...,Too cold to run outside Here s the best wa...,Positive
9,0.270,0.111,Are you aware of this inexpensive therapy that...,Are you aware of this inexpensive therapy that...,Negative


In [13]:
new_df = sentiments_pd[sentiments_pd.Sentiment != 'Neutral']

In [14]:
health_text=new_df.drop(['Positive','Negative','Tweets'],axis=1)


In [15]:
health_text.shape

(5392, 2)

In [18]:
health_text.to_csv("health_text.txt", encoding="utf-8", sep=',',index=False)

In [19]:
health_text.to_csv("health_text.csv", encoding="utf-8", sep=',',index=False)

In [1]:
!jupyter nbconvert --to script Sentiment_Analysis-health.ipynb

[NbConvertApp] Converting notebook Sentiment_Analysis-health.ipynb to script
[NbConvertApp] Writing 2884 bytes to Sentiment_Analysis-health.py
